A Baysian System is implemented in order to assess weather or not you should come to tennis training. Due to the fact that tennis is often played outside, the weather is crucial, especially when you live in a town like London where it rains a lot. Thus, it is difficult to know if you should stay at home a priori or risk to go to training and potentially drive home again due to rain. Also if there was rain the last day or in the morning, the courts might not be playable. 



Enteties: Weather (Sunny, Rain, Cloudy),  Court, Person (how long did you not see each other) 

In [1]:
strange weather -> rainy, cloudy -> playable

1. strange weather = 0.6

2. Rainy

P(Rainy | strange weather) = 0.6
P(Rainy | not strange weather) = 0.1
P(not Rainy | strange weather) = 0.4
P(not Rainy | not strange weather) = 0.9

2. Cloudy

P(Cloudy | strange weather) = 0.9
P(Cloudy | not strange weather) = 0.3
P(not Cloudy | strange weather) = 0.1
P(not Cloudy | not strange weather) = 0.7

3. Playable


P(Playbable | Yes Rainy, Yes Cloudy) = 0.01
P(Playbable | Yes Rainy, No Cloudy) = 0.2
P(Playbable | No Rainy, Yes Cloudy) = 0.8
P(Playbable | No Rainy, No Cloudy) = 0.99



SyntaxError: invalid syntax (<ipython-input-1-4ccff3e0a69c>, line 1)

In [2]:
import numpy as np


In [3]:
from pgmpy.factors.discrete import TabularCPD

In [4]:
from pgmpy.models import BayesianModel

In [25]:
tennis_model = BayesianModel([('Strange Weather', 'Rainy'),
                              ('Strange Weather', 'Cloudy'),
                              ('Rainy', 'Playable'),
                              ('Cloudy', 'Playable')])

In [26]:
strangeweather_cpd = TabularCPD(
    variable = 'Strange Weather',
    variable_card = 2,   # cardinality
    values = [[.6], [.4]])   # ['yes', 'no']

In [27]:
rainy_cpd = TabularCPD(
    variable = 'Rainy',
    variable_card = 2,
    values = [[.7, .1],
              [.3, .9]],
    evidence = ['Strange Weather'],
    evidence_card = [2])

In [28]:
cloudy_cpd = TabularCPD(
    variable = 'Cloudy',
    variable_card = 2,
    values = [[.9, .3],
              [.1, .7]],
    evidence = ['Strange Weather'],
    evidence_card = [2])

In [29]:
playable_cpd = TabularCPD(
    variable = 'Playable',
    variable_card = 2,
    values = [[0.01, .20, .80, 0.99],
              [0.99, .80, .20, 0.01]],
    evidence = ['Rainy', 'Cloudy'],
    evidence_card = [2,2])

In [30]:
tennis_model.add_cpds(strangeweather_cpd, rainy_cpd, cloudy_cpd, playable_cpd)

In [31]:
tennis_model.get_cpds()

[<TabularCPD representing P(Strange Weather:2) at 0x1c41fd96088>,
 <TabularCPD representing P(Rainy:2 | Strange Weather:2) at 0x1c41fce0c48>,
 <TabularCPD representing P(Cloudy:2 | Strange Weather:2) at 0x1c41fcd4408>,
 <TabularCPD representing P(Playable:2 | Rainy:2, Cloudy:2) at 0x1c41fccf908>]

In [32]:
tennis_model.active_trail_nodes('Strange Weather')

{'Strange Weather': {'Cloudy', 'Playable', 'Rainy', 'Strange Weather'}}

In [33]:
tennis_model.local_independencies('Strange Weather')

In [34]:
tennis_model.get_independencies()

(Rainy ⟂ Cloudy | Strange Weather)
(Strange Weather ⟂ Playable | Rainy, Cloudy)
(Playable ⟂ Strange Weather | Rainy, Cloudy)
(Cloudy ⟂ Rainy | Strange Weather)

In [35]:
from pgmpy.inference import VariableElimination

In [37]:
tennis_infer = VariableElimination(tennis_model)

In [41]:
prob_playable = tennis_infer.query(variables = ['Playable'], joint=False)

In [42]:
print(prob_playable['Playable'])

+-------------+-----------------+
| Playable    |   phi(Playable) |
+=============+=================+
| Playable(0) |          0.5012 |
+-------------+-----------------+
| Playable(1) |          0.4988 |
+-------------+-----------------+


In [45]:
prob_rainy = tennis_infer.query(variables = ['Rainy'], joint=False)

In [46]:
print(prob_rainy['Rainy'])

+----------+--------------+
| Rainy    |   phi(Rainy) |
+==========+==============+
| Rainy(0) |       0.4600 |
+----------+--------------+
| Rainy(1) |       0.5400 |
+----------+--------------+


In [47]:
prob_cloudy = tennis_infer.query(variables = ['Cloudy'], joint=False)

In [48]:
print(prob_cloudy['Cloudy'])

+-----------+---------------+
| Cloudy    |   phi(Cloudy) |
+===========+===============+
| Cloudy(0) |        0.6600 |
+-----------+---------------+
| Cloudy(1) |        0.3400 |
+-----------+---------------+


In [50]:
prob_strangeweather = tennis_infer.query(variables = ['Strange Weather'], joint=False)

In [51]:
print(prob_strangeweather['Strange Weather'])

+--------------------+------------------------+
| Strange Weather    |   phi(Strange Weather) |
+====================+========================+
| Strange Weather(0) |                 0.6000 |
+--------------------+------------------------+
| Strange Weather(1) |                 0.4000 |
+--------------------+------------------------+


In [52]:
prob_playable_strangeweather = tennis_infer.query(
        variables = ['Playable'], joint=False,
        evidence = {'Strange Weather':0})

In [53]:
print(prob_playable_strangeweather['Playable'])

+-------------+-----------------+
| Playable    |   phi(Playable) |
+=============+=================+
| Playable(0) |          0.2660 |
+-------------+-----------------+
| Playable(1) |          0.7340 |
+-------------+-----------------+


In [56]:
prob_rainy_strangeweather = tennis_infer.query(
        variables = ['Rainy'], joint=False,
        evidence = {'Strange Weather':0})

In [57]:
print(prob_rainy_strangeweather['Rainy'])

+----------+--------------+
| Rainy    |   phi(Rainy) |
+==========+==============+
| Rainy(0) |       0.7000 |
+----------+--------------+
| Rainy(1) |       0.3000 |
+----------+--------------+


In [58]:
prob_cloudy_strangeweather = tennis_infer.query(
        variables = ['Cloudy'], joint=False,
        evidence = {'Strange Weather':0})

In [59]:
print(prob_cloudy_strangeweather['Cloudy'])

+-----------+---------------+
| Cloudy    |   phi(Cloudy) |
+===========+===============+
| Cloudy(0) |        0.9000 |
+-----------+---------------+
| Cloudy(1) |        0.1000 |
+-----------+---------------+


In [60]:
prob_strangeweather_playable = tennis_infer.query(
        variables = ['Strange Weather'], joint=False,
        evidence = {'Playable':0})

In [61]:
print(prob_strangeweather_playable['Strange Weather'])

+--------------------+------------------------+
| Strange Weather    |   phi(Strange Weather) |
+====================+========================+
| Strange Weather(0) |                 0.3184 |
+--------------------+------------------------+
| Strange Weather(1) |                 0.6816 |
+--------------------+------------------------+


In [62]:
prob_rainy_playable = tennis_infer.query(
        variables = ['Rainy'], joint=False,
        evidence = {'Playable':0})

In [63]:
print(prob_rainy_playable['Rainy'])

+----------+--------------+
| Rainy    |   phi(Rainy) |
+==========+==============+
| Rainy(0) |       0.0357 |
+----------+--------------+
| Rainy(1) |       0.9643 |
+----------+--------------+


In [64]:
prob_cloudy_playable = tennis_infer.query(
        variables = ['Cloudy'], joint=False,
        evidence = {'Playable':0})

In [65]:
print(prob_cloudy_playable['Cloudy'])

+-----------+---------------+
| Cloudy    |   phi(Cloudy) |
+===========+===============+
| Cloudy(0) |        0.4387 |
+-----------+---------------+
| Cloudy(1) |        0.5613 |
+-----------+---------------+


In [67]:
prob_strangeweather_playable_rainy = tennis_infer.query(
        variables = ['Strange Weather'], joint=False,
        evidence = {'Playable':0, 'Rainy':0})

In [68]:
print(prob_strangeweather_playable_rainy['Strange Weather'])

+--------------------+------------------------+
| Strange Weather    |   phi(Strange Weather) |
+====================+========================+
| Strange Weather(0) |                 0.6804 |
+--------------------+------------------------+
| Strange Weather(1) |                 0.3196 |
+--------------------+------------------------+


In [69]:
prob_playable_strangeweather_cloudy = tennis_infer.query(
        variables = ['Cloudy'], joint=False,
        evidence = {'Playable':0, 'Rainy':0})

In [70]:
print(prob_playable_strangeweather_cloudy['Cloudy'])

+-----------+---------------+
| Cloudy    |   phi(Cloudy) |
+===========+===============+
| Cloudy(0) |        0.2179 |
+-----------+---------------+
| Cloudy(1) |        0.7821 |
+-----------+---------------+
